## Casual Analysis of Risk factors associated with depression

In [33]:
import numpy as np
import pandas as pd

import os
from functools import reduce  

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import chi2, f_regression

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error


## Load Data

In [45]:
# full data
full_data = pd.DataFrame()

# from each file
for filename in os.listdir('data'):
    if(filename.endswith(".XPT")):
        file = os.path.join('data', filename)
        newcols = pd.read_sas(file, encoding='UTF-8')
        
        if(len(full_data) == 0):
            full_data = newcols
        else:
            full_data = pd.merge(full_data, newcols, on = "SEQN", how = "outer")

#in case we want the file in csv
full_data.to_csv('data/data_combined.csv')

print(full_data.shape)

(9254, 191)


**3344** rows with values 

In [52]:
cols = full_data.loc[:,"DPQ010":"DPQ100"].columns
valid = full_data.copy()
# valid[cols] = valid[cols].round()

# only care about values 0-3
for i in range(len(cols)):
    valid = valid.loc[valid[cols[i]] <=3]

# sum_dp
valid["sum_DPQ"] = valid[cols].sum(axis="columns", skipna = True)
print(valid.shape)
valid.describe()

(3344, 192)


,SEQN,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,...,WHD080L,WHQ225,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ190,WHQ200,sum_DPQ
count,3344.000000,3344.000000,3308.000000,84.000000,0.0,0.0,0.0,0.0,3307.000000,40.000000,...,22.0,3344.000000,2345.000000,2815.000000,1664.000000,3343.000000,3310.000000,3344.000000,53.000000,3344.000000
mean,98297.694378,1.109450,83.798942,3.023810,NaN,NaN,NaN,NaN,166.386060,2.700000,...,40.0,3.264055,476.912154,482.402487,370.633413,292.377804,637.620846,1.984151,42.509434,5.227572
std,2680.753547,0.465996,23.613383,0.490296,NaN,NaN,NaN,NaN,10.007698,0.723241,...,0.0,1.798048,1686.002440,1765.723266,1712.586305,928.690675,7670.812149,0.124911,11.500205,4.745192
min,93711.000000,1.000000,36.200000,1.000000,NaN,NaN,NaN,NaN,139.700000,1.000000,...,40.0,1.000000,78.000000,70.000000,41.000000,90.000000,12.000000,1.000000,21.000000,1.000000
25%,95951.500000,1.000000,67.300000,3.000000,NaN,NaN,NaN,NaN,159.000000,3.000000,...,40.0,1.000000,145.000000,125.000000,63.000000,160.000000,26.000000,2.000000,35.000000,2.000000
50%,98315.000000,1.000000,79.900000,3.000000,NaN,NaN,NaN,NaN,165.700000,3.000000,...,40.0,4.000000,170.000000,148.000000,66.000000,193.000000,39.000000,2.000000,42.000000,4.000000
75%,100592.500000,1.000000,96.800000,3.000000,NaN,NaN,NaN,NaN,173.600000,3.000000,...,40.0,5.000000,208.000000,180.000000,70.000000,236.000000,54.000000,2.000000,51.000000,7.000000
max,102956.000000,4.000000,219.600000,4.000000,NaN,NaN,NaN,NaN,195.500000,3.000000,...,40.0,9.000000,9999.000000,9999.000000,9999.000000,9999.000000,99999.000000,2.000000,61.000000,28.000000


In [ ]:
# chosen variables TODO: fill these in
feature_names = []